In [ ]:
def search_comedian_instagram(self, comedian_name: str) -> Optional[Dict[str, Any]]:
        """
        Search for comedian's Instagram profile using Google Search API
        
        Args:
            comedian_name: Name of the comedian
            
        Returns:
            Dictionary with instagram_url and follower_count if found, None otherwise
        """
        try:
            # Multiple search queries to increase chances of finding the profile
            search_queries = [
                f"{comedian_name} comedian instagram",
                f"{comedian_name} stand up comedy instagram",
                f"{comedian_name} instagram profile",
                f'"{comedian_name}" instagram comedian'
            ]
            
            for query in search_queries:
                logger.info(f"Searching for: {query}")
                
                params = {
                    'key': self.google_api_key,
                    'cx': self.google_cse_id,
                    'q': query,
                    'num': 10  # Get top 10 results
                }
                
                response = requests.get(self.base_search_url, params=params, timeout=30)
                response.raise_for_status()
                
                data = response.json()
                
                # Look for Instagram URLs and follower counts in the results
                result = self._extract_instagram_data(data, comedian_name)
                
                if result:
                    logger.info(f"Found Instagram for {comedian_name}: {result['instagram_url']}")
                    if result.get('follower_count', 0) > 0:
                        logger.info(f"Found follower count from search: {result['follower_count']}")
                    return result
                
                # Rate limiting
                time.sleep(self.rate_limit_delay)
            
            logger.warning(f"No Instagram found for {comedian_name}")
            return None

In [1]:
import requests
import json

In [ ]:
params = {
                    'key': "API Key here",
                    'cx': "Search Engine ID here",
                    'q': 'dumbfoundead comedian instagram',
                    'num': 5  # Get top 10 results
                }

response = requests.get("https://www.googleapis.com/customsearch/v1", params=params, timeout=30)


data = response.json()

In [21]:
data

{'kind': 'customsearch#search',
 'url': {'type': 'application/json',
  'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'},
 'queries': {'request': [{'title': 'Google Custom Search - Bobby Lee comedian instagram',
    'totalResults': '564000',
    'searchTerms': 'Bobby Lee comedian instagram',
    'count': 5,
    'startIndex': 1,
    'inputEncoding': 'utf8',
    'outputEncoding

In [23]:
data['items'][0]

{'kind': 'customsearch#result',
 'title': 'Bobby Lee (@bobbyleelive) • Instagram photos and videos',
 'htmlTitle': '<b>Bobby Lee</b> (@bobbyleelive) • <b>Instagram</b> photos and videos',
 'link': 'https://www.instagram.com/bobbyleelive/?hl=en',
 'displayLink': 'www.instagram.com',
 'snippet': '2M Followers, 1038 Following, 757 Posts - Bobby Lee (@bobbyleelive) on Instagram: ""',
 'htmlSnippet': '2M Followers, 1038 Following, 757 Posts - <b>Bobby Lee</b> (@bobbyleelive) on <b>Instagram</b>: &quot;&quot;',
 'formattedUrl': 'https://www.instagram.com/bobbyleelive/?hl=en',
 'htmlFormattedUrl': 'https://www.<b>instagram</b>.com/<b>bobbylee</b>live/?hl=en',
 'pagemap': {'xfn': [{}],
  'metatags': [{'og:image': 'https://scontent-atl3-3.cdninstagram.com/v/t51.2885-19/491458955_1882938482537598_4840230306005463840_n.jpg?stp=dst-jpg_s100x100_tt6&_nc_cat=109&ccb=7-5&_nc_sid=bf7eb4&efg=eyJ2ZW5jb2RlX3RhZyI6InByb2ZpbGVfcGljLnd3dy4xMDgwLkMzIn0%3D&_nc_ohc=wnEfUuVdnJwQ7kNvwG6IrAJ&_nc_oc=AdnvCy49BgvqLo

In [30]:
data['items'][0]['pagemap']['metatags'][0]['og:description']

'274K Followers, 4,993 Following, 196 Posts - See Instagram photos and videos from Jonnie Park (@dumbfoundead)'

In [25]:
# Test the og:description approach for extracting follower data
import re

def extract_followers_from_og_description(description):
    """Extract follower count from og:description text"""
    if not description:
        return None
    
    # Common patterns for follower counts
    patterns = [
        r'(\d+(?:\.\d+)?)\s*[mM]\s*[fF]ollowers',  # "2.1M Followers"
        r'(\d+(?:\.\d+)?)[mM]\s*[fF]ollowers',     # "2.1M Followers" (no space)
        r'(\d+(?:,\d+)*)\s*[fF]ollowers',          # "1,234,567 Followers"
        r'(\d+(?:\.\d+)?)\s*[mM]',                 # Just "2.1M"
        r'(\d+(?:,\d+)*)',                         # Any number with commas
    ]
    
    for pattern in patterns:
        match = re.search(pattern, description)
        if match:
            count_str = match.group(1)
            try:
                # Convert to number
                if 'M' in description.upper() or 'm' in description:
                    # Handle millions
                    count = float(count_str.replace(',', '')) * 1000000
                elif 'K' in description.upper() or 'k' in description:
                    # Handle thousands
                    count = float(count_str.replace(',', '')) * 1000
                else:
                    # Handle regular numbers
                    count = int(count_str.replace(',', ''))
                return int(count)
            except ValueError:
                continue
    
    return None

# Test it with the Bobby Lee data
bobby_description = data['items'][0]['pagemap']['metatags'][0]['og:description']
print(f"Description: {bobby_description}")
print(f"Extracted followers: {extract_followers_from_og_description(bobby_description)}")

Description: 2M Followers, 1,038 Following, 757 Posts - See Instagram photos and videos from Bobby Lee (@bobbyleelive)
Extracted followers: 2000000


In [26]:
# Test with all search results to see which ones have follower data
print("All search results with og:description data:")
print("=" * 60)

for i, item in enumerate(data['items']):
    print(f"\nResult {i+1}:")
    print(f"Title: {item.get('title', 'No title')}")
    print(f"Link: {item.get('link', 'No link')}")
    
    # Check if this result has pagemap and metatags
    if 'pagemap' in item and 'metatags' in item['pagemap']:
        metatag = item['pagemap']['metatags'][0]
        og_desc = metatag.get('og:description', '')
        
        if og_desc:
            print(f"og:description: {og_desc}")
            followers = extract_followers_from_og_description(og_desc)
            if followers:
                print(f"🎯 EXTRACTED FOLLOWERS: {followers:,}")
            else:
                print("❌ No follower count found")
        else:
            print("❌ No og:description found")
    else:
        print("❌ No pagemap/metatags found")
    print("-" * 40)

All search results with og:description data:

Result 1:
Title: Bobby Lee (@bobbyleelive) • Instagram photos and videos
Link: https://www.instagram.com/bobbyleelive/?hl=en
og:description: 2M Followers, 1,038 Following, 757 Posts - See Instagram photos and videos from Bobby Lee (@bobbyleelive)
🎯 EXTRACTED FOLLOWERS: 2,000,000
----------------------------------------

Result 2:
Title: dumbfoundead
Link: https://www.facebook.com/dumbfoundead/
og:description: dumbfoundead. 208,955 likes · 15 talking about this. KTOWN, LA 🇰🇷
🎯 EXTRACTED FOLLOWERS: 208,955,000,000
----------------------------------------

Result 3:
Title: Andrew Santino (@cheetosantino) • Instagram photos and videos
Link: https://www.instagram.com/cheetosantino/?hl=en
og:description: 1M Followers, 1,007 Following, 823 Posts - See Instagram photos and videos from Andrew Santino (@cheetosantino)
🎯 EXTRACTED FOLLOWERS: 1,000,000
----------------------------------------

Result 4:
Title: Bobby wishing a thirst bot happy birthday 

In [27]:
# Improved extraction method that focuses on Instagram profiles
def extract_instagram_from_search_results(search_data, comedian_name):
    """
    Extract Instagram URL and follower count from Google search results
    using the og:description method
    """
    if 'items' not in search_data:
        return None
    
    for item in search_data['items']:
        link = item.get('link', '')
        
        # Focus on Instagram URLs only
        if 'instagram.com' not in link:
            continue
            
        # Check for pagemap data
        if 'pagemap' not in item or 'metatags' not in item['pagemap']:
            continue
            
        metatag = item['pagemap']['metatags'][0]
        og_desc = metatag.get('og:description', '')
        
        if not og_desc:
            continue
            
        print(f"🔍 Found Instagram profile: {link}")
        print(f"📝 Description: {og_desc}")
        
        # Extract follower count using Instagram-specific pattern
        import re
        
        # Instagram format: "2M Followers, 1,038 Following, 757 Posts"
        instagram_pattern = r'(\d+(?:\.\d+)?)\s*([MK]?)\s*Followers'
        match = re.search(instagram_pattern, og_desc, re.IGNORECASE)
        
        if match:
            count_str = match.group(1)
            unit = match.group(2).upper()
            
            try:
                count = float(count_str)
                if unit == 'M':
                    count = int(count * 1000000)
                elif unit == 'K':
                    count = int(count * 1000)
                else:
                    count = int(count)
                
                print(f"✅ Extracted {count:,} followers")
                return {
                    'instagram_url': link,
                    'follower_count': count,
                    'source': 'og_description'
                }
            except ValueError:
                print(f"❌ Could not parse follower count: {count_str}")
                continue
        else:
            print(f"❌ No follower pattern found in description")
    
    print(f"❌ No Instagram profile found for {comedian_name}")
    return None

# Test with Bobby Lee data
result = extract_instagram_from_search_results(data, "Bobby Lee")
print(f"\nFinal result: {result}")

🔍 Found Instagram profile: https://www.instagram.com/bobbyleelive/?hl=en
📝 Description: 2M Followers, 1,038 Following, 757 Posts - See Instagram photos and videos from Bobby Lee (@bobbyleelive)
✅ Extracted 2,000,000 followers

Final result: {'instagram_url': 'https://www.instagram.com/bobbyleelive/?hl=en', 'follower_count': 2000000, 'source': 'og_description'}


In [28]:
# Test the updated Instagram extractor
import sys
sys.path.append('.')

from instagram_follower_extractor import InstagramFollowerExtractor

# Initialize the extractor
extractor = InstagramFollowerExtractor()

# Test with the Bobby Lee search data we already have
print("Testing improved extraction method:")
print("=" * 50)

result = extractor._extract_instagram_data(data, "Bobby Lee")
print(f"Result: {result}")

if result:
    print(f"\n✅ SUCCESS!")
    print(f"Instagram URL: {result['instagram_url']}")
    print(f"Follower Count: {result['follower_count']:,}")
    print(f"Source Method: {result.get('source', 'unknown')}")
else:
    print("❌ No result found")

2025-09-19 15:27:44,578 - INFO - Database initialized
2025-09-19 15:27:44,579 - INFO - Instagram Follower Extractor initialized
2025-09-19 15:27:44,579 - INFO - Found follower count in search results: 2000000


Testing improved extraction method:
Result: {'instagram_url': 'https://www.instagram.com/bobbyleelive/', 'follower_count': 2000000, 'search_snippet': '2M Followers, 1038 Following, 757 Posts - Bobby Lee (@bobbyleelive) on Instagram: ""', 'confidence_score': 1.0, 'source': 'og_description'}

✅ SUCCESS!
Instagram URL: https://www.instagram.com/bobbyleelive/
Follower Count: 2,000,000
Source Method: og_description


# Running Instagram Enhancement on Full Dataset

Now let's apply our improved og:description extraction method to the complete Comedy Cellar dataset.

In [31]:
# Load the original comedian dataset
import pandas as pd
import os

# Find the original comedian CSV file
data_dir = "comedy_cellar_data"
csv_files = [f for f in os.listdir(data_dir) if f.endswith('_comedians.csv')]

if csv_files:
    # Use the most recent file
    latest_file = max(csv_files)
    input_file = f"{data_dir}/{latest_file}"
    print(f"Loading data from: {input_file}")
    
    # Load the data
    df = pd.read_csv(input_file)
    print(f"Dataset contains {len(df)} comedian appearances")
    print(f"Unique comedians: {df['comedian_name'].nunique()}")
    
    # Show a sample
    print("\nSample of comedian data:")
    print(df[['comedian_name', 'date', 'venue']].head(10))
else:
    print("No comedian CSV files found in comedy_cellar_data directory")

Loading data from: comedy_cellar_data/test_export_comedians.csv
Dataset contains 90 comedian appearances
Unique comedians: 51

Sample of comedian data:
        comedian_name        date                venue
0           Mike Yard  2024-12-07  Village Underground
1  Harrison Greenbaum  2024-12-07  Village Underground
2         Mia Jackson  2024-12-07  Village Underground
3         Colin Quinn  2024-12-07  Village Underground
4      Ron Bennington  2024-12-07  Village Underground
5       Nicolas Corda  2024-12-07  Village Underground
6          Jon Laster  2024-12-07     MacDougal Street
7    Nathan Macintosh  2024-12-07     MacDougal Street
8      Ron Bennington  2024-12-07     MacDougal Street
9         Mia Jackson  2024-12-07     MacDougal Street


In [32]:
# Let's check for the larger pipeline dataset
import glob

# Look for all CSV files
all_csv_files = glob.glob("**/*comedians*.csv", recursive=True)
print("All comedian CSV files found:")
for file in all_csv_files:
    try:
        temp_df = pd.read_csv(file)
        print(f"{file}: {len(temp_df)} rows, {temp_df['comedian_name'].nunique()} unique comedians")
    except Exception as e:
        print(f"{file}: Error reading - {e}")

# Use the largest dataset
if all_csv_files:
    # Find the file with the most rows
    largest_file = None
    max_rows = 0
    
    for file in all_csv_files:
        try:
            temp_df = pd.read_csv(file)
            if len(temp_df) > max_rows:
                max_rows = len(temp_df)
                largest_file = file
        except:
            continue
    
    if largest_file:
        print(f"\nUsing largest dataset: {largest_file}")
        df = pd.read_csv(largest_file)
        print(f"Final dataset: {len(df)} rows, {df['comedian_name'].nunique()} unique comedians")

All comedian CSV files found:
comedian_data\enhanced_comedians_with_instagram.csv: 110935 rows, 632 unique comedians
comedy_cellar_data\comedy_cellar_recent_2025-03-23_to_2025-09-19_comedians.csv: 18657 rows, 364 unique comedians
comedy_cellar_data\pipeline_20250919_141012_comedians.csv: 110935 rows, 632 unique comedians
comedy_cellar_data\test_export_comedians.csv: 90 rows, 51 unique comedians

Using largest dataset: comedian_data\enhanced_comedians_with_instagram.csv
Final dataset: 110935 rows, 632 unique comedians


In [34]:
# Let's use the original pipeline data to run a fresh enhancement
original_file = "comedy_cellar_data/pipeline_20250919_141012_comedians.csv"
df_original = pd.read_csv(original_file)

print(f"Original dataset: {len(df_original)} rows, {df_original['comedian_name'].nunique()} unique comedians")
print(f"\nColumn names: {list(df_original.columns)}")
print("\nSample of original data:")
print(df_original.head())

# Get unique comedians for Instagram enhancement
unique_comedians = df_original['comedian_name'].unique()
print(f"\nWe need to find Instagram data for {len(unique_comedians)} unique comedians")

Original dataset: 110935 rows, 632 unique comedians

Column names: ['show_id', 'date', 'time', 'venue', 'comedian_id', 'comedian_name', 'order', 'description', 'website', 'thumb_url']

Sample of original data:
   show_id        date     time                venue  comedian_id  \
0    16475  2020-01-01  6:00 pm  Village Underground            0   
1    16475  2020-01-01  6:00 pm  Village Underground            0   
2    16475  2020-01-01  6:00 pm  Village Underground            0   
3    16475  2020-01-01  6:00 pm  Village Underground            0   
4    16475  2020-01-01  6:00 pm  Village Underground            0   

         comedian_name  order  \
0            Mike Yard      0   
1    Vladimir  Caamano      1   
2  Casey James Salengo      2   
3            Joe Machi      3   
4         Lenny Marcus      4   

                                         description  \
0  FROM "NIGHTLY SHOW WITH LARRY WILMORE", BAD BO...   
1  FROM  CBS's "TOMMY", FROM JIMMY KIMMEL LIVE, H...   
2  From 

In [35]:
# Let's test our enhanced extractor on a small sample first
sample_comedians = ['Mike Yard', 'Joe Machi', 'Amy Schumer', 'Dave Chappelle', 'Jim Gaffigan']

print("Testing Instagram extraction on sample comedians:")
print("=" * 60)

results = []
for comedian in sample_comedians:
    print(f"\n🎭 Processing: {comedian}")
    
    result = extractor.search_comedian_instagram(comedian)
    
    if result:
        print(f"✅ Found Instagram: {result['instagram_url']}")
        if result.get('follower_count', 0) > 0:
            print(f"📊 Followers: {result['follower_count']:,}")
        results.append({
            'comedian': comedian,
            'instagram_url': result['instagram_url'],
            'follower_count': result.get('follower_count', 0),
            'source': result.get('source', 'unknown')
        })
    else:
        print(f"❌ No Instagram found")
        results.append({
            'comedian': comedian,
            'instagram_url': None,
            'follower_count': 0,
            'source': 'not_found'
        })
    
    # Small delay to respect rate limits
    import time
    time.sleep(1)

print(f"\n\nSample Results Summary:")
print("=" * 40)
for r in results:
    status = "✅" if r['instagram_url'] else "❌"
    followers = f"{r['follower_count']:,}" if r['follower_count'] > 0 else "0"
    print(f"{status} {r['comedian']}: {followers} followers ({r['source']})")

2025-09-19 15:30:18,608 - INFO - Searching for: Mike Yard comedian instagram


Testing Instagram extraction on sample comedians:

🎭 Processing: Mike Yard


2025-09-19 15:30:19,511 - INFO - Found follower count in search results: 15000
2025-09-19 15:30:19,511 - INFO - Found Instagram for Mike Yard: https://www.instagram.com/mikeyardcomedy/
2025-09-19 15:30:19,517 - INFO - Found follower count from search: 15000


✅ Found Instagram: https://www.instagram.com/mikeyardcomedy/
📊 Followers: 15,000


2025-09-19 15:30:20,522 - INFO - Searching for: Joe Machi comedian instagram


...

✅ Found Instagram: https://www.instagram.com/davechappelle/
📊 Followers: 3,000,000


2025-09-19 15:30:26,126 - INFO - Searching for: Jim Gaffigan comedian instagram



🎭 Processing: Jim Gaffigan


2025-09-19 15:30:26,952 - INFO - Found follower count in search results: 2000000
2025-09-19 15:30:26,953 - INFO - Found Instagram for Jim Gaffigan: https://www.instagram.com/jimgaffigan/
2025-09-19 15:30:26,953 - INFO - Found follower count from search: 2000000


✅ Found Instagram: https://www.instagram.com/jimgaffigan/
📊 Followers: 2,000,000


Sample Results Summary:
✅ Mike Yard: 15,000 followers (og_description)
✅ Joe Machi: 44,000 followers (og_description)
✅ Amy Schumer: 13,000,000 followers (og_description)
✅ Dave Chappelle: 3,000,000 followers (og_description)
✅ Jim Gaffigan: 2,000,000 followers (og_description)


# Running Full Dataset Enhancement

Perfect! Our improved og:description method is working flawlessly. Now let's run it on the complete dataset using the command-line tool.

In [36]:
# Run the full Instagram enhancement process
import subprocess
import sys

# The command to run the full enhancement
input_file = "comedy_cellar_data/pipeline_20250919_141012_comedians.csv"
output_file = "comedian_data/enhanced_comedians_with_instagram_v2.csv"

command = [
    sys.executable, 
    "enhance_with_instagram.py", 
    "--input", input_file,
    "--output", output_file
]

print("Starting full Instagram enhancement process...")
print(f"Input: {input_file}")
print(f"Output: {output_file}")
print(f"Command: {' '.join(command)}")
print("\nThis will take approximately 20-30 minutes for 632 comedians...")
print("Press Ctrl+C if you want to interrupt the process.")

# Note: Running this in the notebook will take a long time
# It's better to run it in the terminal
print("\n⚠️  For better performance, run this command in the terminal:")
print(f"python enhance_with_instagram.py --input \"{input_file}\" --output \"{output_file}\"")

# Uncomment the next line to actually run it (will take 20-30 minutes)
# result = subprocess.run(command, capture_output=True, text=True)

Starting full Instagram enhancement process...
Input: comedy_cellar_data/pipeline_20250919_141012_comedians.csv
Output: comedian_data/enhanced_comedians_with_instagram_v2.csv
Command: c:\Users\schue\OneDrive\Dokumente\Data Science\Comedy Project\.venv\Scripts\python.exe enhance_with_instagram.py --input comedy_cellar_data/pipeline_20250919_141012_comedians.csv --output comedian_data/enhanced_comedians_with_instagram_v2.csv

This will take approximately 20-30 minutes for 632 comedians...
Press Ctrl+C if you want to interrupt the process.

⚠️  For better performance, run this command in the terminal:
python enhance_with_instagram.py --input "comedy_cellar_data/pipeline_20250919_141012_comedians.csv" --output "comedian_data/enhanced_comedians_with_instagram_v2.csv"


In [ ]:
# Monitor the enhancement process and show current results
import time
import os

# Check if we already have enhanced data
existing_file = "comedian_data/enhanced_comedians_with_instagram.csv"
if os.path.exists(existing_file):
    print("📊 CURRENT ENHANCED DATASET AVAILABLE")
    print("=" * 50)
    
    df_enhanced = pd.read_csv(existing_file)
    
    # Calculate statistics
    total_appearances = len(df_enhanced)
    unique_comedians = df_enhanced['comedian_name'].nunique()
    with_instagram = df_enhanced['instagram_url'].notna().sum()
    with_followers = df_enhanced['follower_count'].fillna(0).gt(0).sum()
    
    print(f"📈 Total comedian appearances: {total_appearances:,}")
    print(f"🎭 Unique comedians: {unique_comedians:,}")
    print(f"📱 Appearances with Instagram: {with_instagram:,}")
    print(f"👥 Appearances with follower data: {with_followers:,}")
    print(f"🎯 Instagram discovery rate: {(with_instagram/total_appearances)*100:.1f}%")
    
    # Show top comedians by followers
    top_comedians = df_enhanced.groupby('comedian_name').agg({
        'follower_count': 'first',
        'instagram_url': 'first'
    }).sort_values('follower_count', ascending=False).head(10)
    
    print(f"\n🔥 TOP 10 COMEDIANS BY INSTAGRAM FOLLOWERS:")
    print("-" * 50)
    for i, (name, data) in enumerate(top_comedians.iterrows(), 1):
        followers = data['follower_count']
        if followers > 0:
            print(f"{i:2d}. {name:<25} {followers:>10,} followers")
else:
    print("⏳ Enhancement process is running...")
    print("Check terminal output for progress updates.")

In [6]:
len(data)

6